## 多个feas_iti且最后一趟列车id相同的乘客

|           |    rid | iti_id |    path_id | seg_id | train_id | board_ts | alight_ts |
|:----------|-------:|-------:|-----------:|-------:|---------:|---------:|----------:|
|93194730   | 1525794    |   2 | 1076103701  |     2 | 10301630   |  65071   |   65670|
|8610761    |  119014    |   6 | 1101102701  |     3 | 11002833   |  30045   |   30791|
|18578209   |  230428    |   6 | 1053108503  |     2 | 10401849   |  29715   |   30255|
|72855273   | 1321497    |   3 | 1110112802  |     1 | 10401844   |  60803   |   61537|
|105488240  | 1781873    |   2 | 1028106602  |     2 | 10401738   |  69396   |   69625|
|90908613   | 1485498    |   2 | 1023112302  |     2 | 10702370   |  64374   |   64780|
|91388272   | 1494286    |   1 | 1104111501  |     2 | 10702247   |  65070   |   65278|
|53621559   |  957712    |   6 | 1121102503  |     3 | 10401786   |  49174   |   50141|
|19767473   |  243641    |   1 | 1081103901  |     2 | 10702323   |  29509   |   31141|
|107978876  | 1880069    |   2 | 1129107002  |     1 | 10201226   |  73278   |   74154|
|106615427  | 1817923    |   1 | 1027110801  |     2 | 10301362   |  71471   |   73514|
|76737269   | 1359494    |   1 | 1099102101  |     2 | 10301421   |  64071   |   65266|
|100422001  | 1661221    |   5 | 1061108702  |     1 | 10100270   |  66558   |   66886|
|2408620    |   41409    |   2 | 1111106502  |     1 | 10401886   |  25727   |   25836|
|101105912  | 1675451    |   1 | 1110101301  |     1 | 10401914   |  66944   |   67031|
|92513072   | 1514419    |   3 | 1097110302  |     3 | 10200839   |  65302   |   65408|
|81121188   | 1412555    |   1 | 1061109504  |     2 | 10201199   |  63371   |   63480|
|108069499  | 1886032    |   1 | 1032108201  |     1 | 10200950   |  73568   |   75192|
|73737366   | 1331466    |   1 | 1086110901  |     2 | 10702414   |  61616   |   62107|
|48046191   |  811513    |   3 | 1076103602  |     3 | 10301265   |  42747   |   43084|

In [44]:
from tkinter.messagebox import showerror

import numpy as np

from src import config
from src.utils import *
from src.globals import *

config.load_config()

assigned = read_all(config.CONFIG["results"]["assigned"])
print(assigned.shape)
assigned

[INFO] read_all(('assigned.pkl',), {}) executing at 04-24 16:17:50.
[INFO] Loaded 1 versioned files for assigned.pkl
[INFO] read_all(('assigned.pkl',), {}) executed in  04-24 16:17:50. (04-24 16:17:50 -> 0.0151s )
(906242, 7)


,rid,iti_id,path_id,seg_id,train_id,board_ts,alight_ts
0,132,1,1127110101,1,10200773,22740,23821
1,284,1,1027103801,1,11002637,20292,21355
2,308,1,1132107101,1,10401755,22659,24066
3,312,1,1101103301,1,10200756,20219,22237
4,313,1,1101103301,1,10200756,20219,22237
...,...,...,...,...,...,...,...
906237,2047044,1,1055100801,1,10402102,85925,86264
906238,2047046,1,1091111301,1,10401899,85983,86200
906239,2047049,1,1025111301,1,10401899,86095,86200
906240,2047059,1,1114105701,1,10702494,86172,86255


In [47]:
# ① 删除所有仅包含一个seg_id的(rid, iti_id)组合
seg_count = assigned.groupby(['rid', 'iti_id'])['seg_id'].transform('nunique')
assigned = assigned[seg_count > 1]

# ② 找到所有seg_id > 1的行index，并将前一行的seg_id, board_ts, alight_ts合并到当前行右侧
# 首先，我们标识出需要操作的行
assigned['next_index'] = assigned.groupby(['rid', 'iti_id'])['seg_id'].shift(-1).notna()

assigned["next_board_ts"] = assigned["board_ts"].shift(-1)
assigned = assigned[assigned["next_index"]]
assigned["transfer_time"] = assigned["next_board_ts"] - assigned["alight_ts"]
assigned

,rid,iti_id,path_id,seg_id,train_id,board_ts,alight_ts,next_index,next_board_ts,transfer_time
5,321,1,1101103801,1,10200756,20219,21294,True,21601.0,307.0
21,375,1,1082111601,1,10200773,20652,20999,True,21426.0,427.0
23,376,1,1082105801,1,10200773,20652,20999,True,21426.0,427.0
27,379,1,1082111501,1,10200773,20652,20999,True,21426.0,427.0
29,380,1,1082105801,1,10200773,20652,20999,True,21426.0,427.0
...,...,...,...,...,...,...,...,...,...,...
905830,2046282,1,1018101201,1,10100421,83847,84544,True,84717.0,173.0
905842,2046296,1,1133110102,1,10402126,83821,83912,True,84259.0,347.0
905877,2046354,1,1033109601,1,10201237,84217,84324,True,84561.0,237.0
905884,2046360,1,1042111601,1,10100421,83965,84544,True,84717.0,173.0


In [48]:
data = assigned[["path_id", "seg_id", "alight_ts", "transfer_time"]].values.astype(int)
print(data)

[[1101103801          1      21294        307]
 [1082111601          1      20999        427]
 [1082105801          1      20999        427]
 ...
 [1033109601          1      84324        237]
 [1042111601          1      84544        173]
 [1033101601          1      84324        237]]


In [23]:
from src.globals import get_k_pv

k_pv_ = get_k_pv()[:, :-2]
print(f"k_pv_:\n{k_pv_[:5]}")
k_pv = k_pv_[k_pv_[:, -1] != "in_vehicle"]
# print(k_pv)
# print(np.where(k_pv[:, 1] == 1)[0][1:] - 1)
last_seg_idx = np.where(k_pv[:, 1] == 1)[0][1:] - 1
mask = np.ones(len(k_pv), dtype=bool)
mask[last_seg_idx] = False
mask[-1] = False
# print(k_pv[mask & (k_pv[:, 1] != 1)])
k_pv = k_pv[mask & (k_pv[:, 1] != 1)]  # only transfer links

# platform swap
k_pv_swap = k_pv[k_pv[:, -1] == "platform_swap"][:, :-1]
print("k_pv_swap: ")
print(k_pv_swap[:5])  # path_id, pv_id, platform_id1, platform_id2

# egress-entry
k_pv_ee = k_pv[k_pv[:, -1] != "platform_swap"]
k_pv_ee = np.hstack((k_pv_ee[::2, :-2], k_pv_ee[1::2, [3]]))
print("k_pv_ee: ")
print(k_pv_ee[:5])  # path_id, pv_id1, platform_id1, platform_id2

k_pv_:
[[1001100201 1 1001 101200 'entry']
 [1001100201 2 101200 101300 'in_vehicle']
 [1001100201 3 101300 1002 'egress']
 [1001100301 1 1001 101200 'entry']
 [1001100301 2 101200 101470 'in_vehicle']]
k_pv_swap: 
[[1001100502 6 104291 102321]
 [1001100803 6 102321 104291]
 [1001100903 6 102321 104291]
 [1001101303 6 102321 104291]
 [1001101304 6 104291 102321]]
k_pv_ee: 
[[1001100501 3 101220 107211]
 [1001100501 6 107461 102291]
 [1001100502 3 101250 104311]
 [1001100503 3 101260 102351]
 [1001100601 3 101260 102350]]


In [72]:
# get seg_id in k_pv
k_pv_ = get_k_pv()[:, :-2]
df = pd.DataFrame(k_pv_, columns=["path_id", "pv_id", "node1", "node2", "link_type"])
in_vehicle_df = df[df['link_type'] == 'in_vehicle'].copy()
in_vehicle_df['seg_id'] = in_vehicle_df.groupby('path_id').cumcount() + 1
df = pd.merge(df, in_vehicle_df[['seg_id']], left_index=True, right_index=True, how='left')
df['seg_id'] = df['seg_id'].fillna(0).astype(int)

k_pv_ = df.values  # path_id, pv_id, node1, node2, link_type, seg_id
print(k_pv_.shape)

path = read_("path")
path_id_with_transfer = path[path["transfer_cnt"] > 0]["path_id"].values
k_pv_ = k_pv_[np.isin(k_pv_[:, 0], path_id_with_transfer)]

# remove real entry and egress links
last_seg_idx = np.where(k_pv_[:, 1] == 1)[0][1:] - 1
mask = np.ones(len(k_pv_), dtype=bool)
mask[last_seg_idx] = False
mask[-1] = False
k_pv_ = k_pv_[mask & (k_pv_[:, 1] != 1)]

# egress_rows = np.where(k_pv_[:, -2] == "egress")[0]
# entry_rows = np.where(k_pv_[:, -2] == "entry")[0]
# swap_rows = np.where(k_pv_[:, -2] == "platform_swap")[0]


print(k_pv_[:10])

[INFO] read_(('path',), {}) executing at 04-24 17:00:16.
[INFO] Reading file: results\path\path.pkl
[INFO] read_(('path',), {}) executed in  04-24 17:00:16. (04-24 17:00:16 -> 0.0100s )


KeyboardInterrupt: 

In [106]:
# add seg_id in k_pv
k_pv_ = get_k_pv()[:, :-2]
df = pd.DataFrame(k_pv_, columns=["path_id", "pv_id", "node1", "node2", "link_type"])
in_vehicle_df = df[df['link_type'] == 'in_vehicle'].copy()
in_vehicle_df['seg_id'] = in_vehicle_df.groupby('path_id').cumcount() + 1
df = pd.merge(df, in_vehicle_df[['seg_id']], left_index=True, right_index=True, how='left')

# delete non-transfer paths
path = read_("path", show_timer=False)
path_id_with_transfer = path[path["transfer_cnt"] > 0]["path_id"].values
df = df[df['path_id'].isin(path_id_with_transfer)]

# delete first and last pathvia row
first_pv_ind = (df["pv_id"] == 1)  # entry
last_pv_ind = first_pv_ind.shift(-1)  # egress
df = df[~(first_pv_ind | last_pv_ind)].iloc[:-1, :]

# aggregate egress-entry transfer link
df["seg_id"] = df["seg_id"].ffill().astype(int)
df["next_node2"] = df["node2"].shift(-1)
# fix swap expressions
df.loc[df["link_type"] == "platform_swap", "next_node2"] = df["node2"]
df.loc[df["link_type"] == "egress", "link_type"] = "egress-entry"
res = df[df["link_type"].isin(["egress-entry", "platform_swap"])][["path_id", "seg_id", "node1", "next_node2", "link_type"]].values
print(res[:10])

[INFO] Reading file: results\path\path.pkl
[[1001100501 1 101220 107211 'egress-entry']
 [1001100501 2 107461 102291 'egress-entry']
 [1001100502 1 101250 104311 'egress-entry']
 [1001100502 2 104291 102321 'platform_swap']
 [1001100503 1 101260 102351 'egress-entry']
 [1001100601 1 101260 102350 'egress-entry']
 [1001100601 2 102360 103300 'egress-entry']
 [1001100602 1 101250 104310 'egress-entry']
 [1001100602 2 104330 103290 'egress-entry']
 [1001100603 1 101220 107210 'egress-entry']]


In [110]:
df = pd.DataFrame(res, columns=["path_id", "pv_id", "node1", "node2", "transfer_type"])
print(df.groupby(["node1", "node2"]).size())

node1   node2 
101220  107210     175
        107211     124
101221  107210    1505
        107211     896
101250  104310     624
                  ... 
107461  102290     859
        102291     544
110211  103371    1053
        107400     590
        107401     595
Length: 116, dtype: int64


In [78]:
# get seg_id in k_pv
k_pv_ = get_k_pv()[:, :-2]
df = pd.DataFrame(k_pv_, columns=["path_id", "pv_id", "node1", "node2", "link_type"])
in_vehicle_df = df[df['link_type'] == 'in_vehicle'].copy()
in_vehicle_df['seg_id'] = in_vehicle_df.groupby('path_id').cumcount() + 1
df = pd.merge(df, in_vehicle_df[['seg_id']], left_index=True, right_index=True, how='left')
df['seg_id'] = df['seg_id'].fillna(0).astype(int)

k_pv_ = df.values  # path_id, pv_id, node1, node2, link_type, seg_id

# remove non-transfer paths
path_id_list, seg_counts = np.unique(k_pv_[:, 0], return_counts=True)
path_id_with_transfers = path_id_list[seg_counts > 3]
# k_pv_ = k_pv_[np.isin(k_pv_[:, 0], path_id_with_transfers)]
set(path_id_with_transfers)

{1033109504,
 1033109505,
 1114112002,
 1114112004,
 1041104901,
 1041104902,
 1041104903,
 1122107401,
 1049100301,
 1049100302,
 1049100303,
 1006108701,
 1006108702,
 1006108703,
 1006108704,
 1006108705,
 1006108706,
 1006108707,
 1014104101,
 1014104102,
 1014104103,
 1014104104,
 1014104105,
 1095106601,
 1095106602,
 1103102001,
 1103102002,
 1103102003,
 1103102004,
 1103102005,
 1060110401,
 1060110402,
 1060110403,
 1060110404,
 1060110405,
 1068105801,
 1068105802,
 1068105803,
 1076101201,
 1076101202,
 1076101203,
 1076101204,
 1076101205,
 1076101206,
 1076101207,
 1076101208,
 1033109601,
 1033109602,
 1033109603,
 1041105001,
 1122107501,
 1122107502,
 1122107503,
 1122107504,
 1049100401,
 1049100402,
 1049100403,
 1122107505,
 1122107506,
 1122107507,
 1130102901,
 1006108801,
 1006108802,
 1006108803,
 1006108804,
 1006108805,
 1014104201,
 1014104202,
 1014104203,
 1014104204,
 1014104205,
 1095106701,
 1103102101,
 1103102102,
 1103102103,
 1103102104,
 1103102105,

In [17]:
# 转换为 DataFrame
df = pd.DataFrame(k_pv_, columns=["path_id", "pv_id", "node1", "node2", "link_type"])

# 仅保留'in_vehicle'类型的行
in_vehicle_df = df[df['link_type'] == 'in_vehicle'].copy()

# 使用groupby和rank为每个path_id分配顺序编号
in_vehicle_df['seg_id'] = in_vehicle_df.groupby('path_id').cumcount() + 1

In [21]:
# get seg_id in k_pv
k_pv_ = get_k_pv()[:, :-2]
df = pd.DataFrame(k_pv_, columns=["path_id", "pv_id", "node1", "node2", "link_type"])
in_vehicle_df = df[df['link_type'] == 'in_vehicle'].copy()
in_vehicle_df['seg_id'] = in_vehicle_df.groupby('path_id').cumcount() + 1
df = pd.merge(df, in_vehicle_df[['seg_id']], left_index=True, right_index=True, how='left')
df['seg_id'] = df['seg_id'].fillna(0).astype(int)



31.6 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
122 ms ± 1.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
